In [1]:
from abc import ABC,abstractclassmethod

### Interfaces

In [2]:
class MyInterface(ABC):
    @abstractclassmethod
    def my_method(self):
        pass

In [3]:
class MyClass(MyInterface):
    def __init__(self):
        pass
    def my_method(self):
        print("my_method implementation")

In [4]:
my_class = MyClass()
my_class.my_method()

my_method implementation


In [5]:
def process_my_interface(obj: MyInterface):
    obj.my_method()

In [6]:
process_my_interface(my_class)

my_method implementation


### Abstract classes

In [7]:
class Shape(ABC):
    def __init__(self,color):
        self.color = color
    @abstractclassmethod
    def area(self):
        pass
    @abstractclassmethod
    def perimeter(self):
        pass

In [8]:
class Rectangle(Shape):
    def __init__(self, width, height, color):
        super().__init__(color)
        self.width = width
        self.height = height
    def area(self):
        return self.width * self.height
    def perimeter(self):
        return 2*(self.width + self.height)

In [9]:
rectangle = Rectangle(10, 25, "black")
print(rectangle.area())

250


### Encapsulation

In [10]:
class BankAccount:
    def __init__(self,account_number,balance):
        self._account_number = account_number  # protected
        self.__balance = balance # private

In [11]:
bank_account = BankAccount("0001ssee3s2332", 123465)
print(bank_account._account_number) # not recommended


0001ssee3s2332


### Singleton

#### (not thread-safety)

In [12]:
class ClassicSingleton:
    # class level variable
    _instance = None
    # override the method to control initialization
    def __init__(self):
        raise RunTimeError("Call instance()")
    @classmethod
    def get_instance(cls): # lazy loading
        if not cls._instance:
            # create new instance of the class
            cls._instance = cls.__new__(cls)
        return cls._instance
            

In [13]:
class Singleton:
    # class level variable
    _instance = None
    #override and control how new objects are created
    def __new__(cls):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance
    

In [14]:
class SingletonMeta(type):
    # dictionary stores single instance of the class
    _instances = {}
    # override: called during creation of sub-types
    def  __init__(cls,name,bases,dct):
        super().__init__(name,bases,dct)
        #eager loading  of the class instance
        cls._instances = super().__call__()
    # return the singleton instance
    def __call__(cls, *args,**kwargs):
        return cls._instances[cls]
        

In [15]:
s1 = ClassicSingleton.get_instance()
s2 = ClassicSingleton.get_instance()
print(s1 is s2)
print(s1)
print(s2)

True


## LOGGER

In [16]:
from abc import ABCMeta, abstractmethod
import threading
import logging

# This is a metaclass for creating singleton classes. Singleton classes allow only one instance.
class SingletonMeta(type):
    _instances = {}  # Dictionary to hold the instance reference for each class.
    _lock = threading.Lock()  # A lock to ensure thread-safe singleton instantiation.

    def __call__(cls, *args, **kwargs):
        # Acquire the lock to make sure that only one thread can enter this block at a time.
        with cls._lock:
            # Check if the instance already exists for the class.
            if cls not in cls._instances:
                # If not, create the instance and store it in the _instances dictionary.
                cls._instances[cls] = super().__call__(*args, **kwargs)
        # Return the instance.
        return cls._instances[cls]

# This metaclass combines the features of SingletonMeta and ABCMeta.
class SingletonABCMeta(ABCMeta, SingletonMeta):
    def __new__(cls, name, bases, namespace):
        # Create a new class using the combined metaclasses.
        return super().__new__(cls, name, bases, namespace)

# BaseLogger is an abstract class with the SingletonABCMeta metaclass.
class BaseLogger(metaclass=SingletonABCMeta):
    # These methods are abstract, meaning subclasses must implement these methods.
    @abstractmethod
    def debug(cls, message: str):
        pass

    @abstractmethod
    def info(cls, message: str):
        pass

    @abstractmethod
    def warning(cls, message: str):
        pass

    @abstractmethod
    def error(cls, message: str):
        pass

    @abstractmethod
    def critical(cls, message: str):
        pass

# MyLogger is a concrete implementation of BaseLogger.
class MyLogger(BaseLogger):
    def __init__(self):
        print('<Logger init> initializing logger...')
        # Create a logger object with the specified name.
        self._logger = logging.getLogger('my_logger')
        # Set the logging level to DEBUG.
        self._logger.setLevel(logging.DEBUG)

        # Create a file handler to log messages to a file.
        file_handler = logging.FileHandler('my_log_file.log')
        # Set the file handler logging level to DEBUG.
        file_handler.setLevel(logging.DEBUG)

        # Create a console handler to log messages to the console.
        console_handler = logging.StreamHandler()
        # Set the console handler logging level to INFO.
        console_handler.setLevel(logging.INFO)

        # Define the log message format.
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        # Set the formatter for both the file and console handlers.
        file_handler.setFormatter(formatter)
        console_handler.setFormatter(formatter)

        # Add the file and console handlers to the logger.
        self._logger.addHandler(file_handler)
        self._logger.addHandler(console_handler)

    # Implementations of the abstract methods in BaseLogger.
    def debug(self, message: str):
        self._logger.debug(message)

    def info(self, message: str):
        self._logger.info(message)

    def warning(self, message: str):
        self._logger.warning(message)

    def error(self, message: str):
        self._logger.error(message)

    def critical(self, message: str):
        self._logger.critical(message)


# Create an instance of MyLogger.
logger = MyLogger()
# Log different types of messages.
logger.debug('This is a debug message')
logger.info('This is an info message')
logger.warning('This is a warning message')
logger.error('This is an error message')
logger.critical('This is a critical message')


2024-02-03 10:06:46,637 - my_logger - INFO - This is an info message
2024-02-03 10:06:46,637 - my_logger - WARNING - This is a warning message
2024-02-03 10:06:46,638 - my_logger - ERROR - This is an error message
2024-02-03 10:06:46,638 - my_logger - CRITICAL - This is a critical message


<Logger init> initializing logger...


### Factory 

In [40]:
from abc import ABC, abstractmethod
from enum import Enum

# Step 0: Create an enumeration for vehicle types
class VehicleType(Enum):
    CAR = "Car"
    MOTORCYCLE = "Motorcycle"
    BICYCLE = "Bicycle"

# Step 1: Create an abstract Vehicle class
class Vehicle(ABC):
    @abstractmethod
    def get_name(self) -> str:
        pass

# Step 2: Create concrete vehicle classes
class Car(Vehicle):
    def get_name(self) -> str:
        return VehicleType.CAR.value

class Motorcycle(Vehicle):
    def get_name(self) -> str:
        return VehicleType.MOTORCYCLE.value

class Bicycle(Vehicle):
    def get_name(self) -> str:
        return VehicleType.BICYCLE.value

# Step 3: Create a VehicleFactory class
class VehicleFactory:
    def create_vehicle(self, vehicle_type: VehicleType) -> Vehicle:               
        if vehicle_type.value == 'Car':
            return Car()
        elif vehicle_type.value == 'Motorcycle':
            return Motorcycle()
        else:
            return Bicycle()


vehicle_factory = VehicleFactory()


car = vehicle_factory.create_vehicle(VehicleType.CAR)
print(car.get_name())
motorcycle = vehicle_factory.create_vehicle(VehicleType.MOTORCYCLE)
print(motorcycle.get_name())

bicycle = vehicle_factory.create_vehicle(VehicleType.BICYCLE)
print(bicycle.get_name())



Car
Motorcycle
Bicycle
